In [1]:
SEED = 1412
Test_Ratio = 0.2

import os 
import json 
import pandas as pd

Home = os.getcwd()
DataFolder = 'Data'
DataFolder = os.path.join(Home,DataFolder)

MainResultFolder = 'Main_Result'
try:
    os.mkdir(MainResultFolder)
except:
    print('Already Got this Data Folder too')
MainResultFolder = os.path.join(Home,MainResultFolder)

Supplementary_result = 'Supplementary_Result'
try:
    os.mkdir(Supplementary_result)
except:
    print('High on Energy')
Supplementary_result = os.path.join(Home,Supplementary_result)

Train_Folder = os.path.join(DataFolder,'Train.csv')
Test_Folder = os.path.join(DataFolder,'Test.csv')

Train_File = os.path.join(DataFolder,'Train.csv')
Test_File = os.path.join(DataFolder,'Test.csv')

Train = pd.read_csv(Train_File)
Test = pd.read_csv(Test_File)
col_lst = list(Test.columns)
col_lst.remove('Answers')
col_lst = [i for i in col_lst if not i.startswith('Unnamed')]
X_train = Train[col_lst]
X_test = Test[col_lst]
y_train = Train['Answers']
y_test = Test['Answers']

#print(X_train,y_train,X_test,y_test,sep='\n')

import json
import numpy as np 
import niapy
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

clf.fit(X_train, y_train)

val = clf.score(X_test, y_test)

print(val)

from niapy.algorithms.basic import GeneticAlgorithm

Algo = GeneticAlgorithm()
#Algo.set_parameters(population_size=population_size,tournament_size=tournament_size,mutation_rate=mutation_rate,crossover_rate=crossover_rate,selection=selection,crossover=crossover,mutation=mutation,seed=SEED) 

nia_mdl = NatureInspiredSearchCV(
    estimator=RandomForestClassifier(),#(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
    param_grid={},
    algorithm=Algo,
    runs=1,
)


Already Got this Data Folder too
High on Energy
0.938220378054403


In [6]:
from niapy.algorithms.basic import DifferentialEvolution
from niapy.task import Task

# we will run 10 repetitions of Weighed, velocity clamped PSO on the Pinter problem
for i in range(10):
    task = Task(problem='Sphere', dimension=10, max_evals=10000)
    algorithm = DifferentialEvolution()
    best_x, best_fit = algorithm.run(task)
    print(best_fit)

1.2015085342297442
0.4334870641513088
1.6115507092779044
0.4449862098404856
0.5284102971007305
1.6245354202544036
1.6802736323141467
1.7421651009341452
1.2548406250044593
1.9422910026540448


In [ ]:
import numpy as np
from niapy.model_selection_param_grid import ParamGrid
from niapy.model_selection_parameter_search import ParameterSearch
from niapy.model_selection_stagnation_stopping_task import StagnationStoppingTask

from niapy.algorithms.basic import FireflyAlgorithm, BatAlgorithm, GreyWolfOptimizer
from niapy.algorithms.modified import HybridBatAlgorithm, HybridSelfAdaptiveBatAlgorithm

from sklearn.model_selection._search import BaseSearchCV, check_cv, is_classifier

class NatureInspiredSearchCV(BaseSearchCV):
    def fit(self, X, y=None, *, groups=None, **fit_params):
        self.cv_orig = check_cv(self.cv, y, classifier=is_classifier(self.estimator))
        self.n_splits = self.cv_orig.get_n_splits(X, y, groups)
        return super().fit(X, y, groups=groups, **fit_params)

    def _run_search(self, evaluate_candidates):
        self.__param_grid = ParamGrid(self.param_grid)
        self.__algorithm = self.algorithm
        self.__print_run_search_log()
        problem = ParameterSearch(evaluate_candidates, self.__param_grid)

        self.optimization_logs_ = []

        for i in range(1, self.runs + 1):
            def iteration_finished_callback(iteration, stagnation_gen_count):
                stagnation = f' | Stagnation {stagnation_gen_count}'

                if self.max_stagnating_gen > 1:
                    stagnation = f'{stagnation}/{self.max_stagnating_gen}'

                if self.verbose > 1:
                    print(f'Run {i}/{self.runs}, Iteration {iteration}/{self.max_n_gen}{stagnation}'
                          f' | There were {problem.evaluation_count} candidates trained until now')

            task = StagnationStoppingTask(
                max_iters=self.max_n_gen,
                max_stagnating_gen=self.max_stagnating_gen,
                problem=problem,
                iteration_finished_callback=iteration_finished_callback
            )

            self.__algorithm.run(task=task)

            if self.__algorithm.exception:
                raise self.__algorithm.exception

            # invert scores in the optimization logs
            self.optimization_logs_.append([(log[0], log[1] * -1) for log in task.optimization_logs_])

            if self.verbose > 0:
                print(f'Run {i}/{self.runs} finished')

        candidates = problem.evaluation_count

        if self.cv is None:
            print(f'Optimization finished, {candidates} candidates were fitted')
        else:
            print(f'Optimization finished, {candidates} candidates were fitted '
                  f'(totalling {candidates * self.n_splits} fits)')

    def __init__(self, estimator, param_grid, algorithm='hba', population_size=50, max_n_gen=100, runs=3,
                 max_stagnating_gen=20, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0,
                 random_state=None,
                 pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=True):
        super().__init__(estimator, scoring=scoring, n_jobs=n_jobs, refit=refit, cv=cv, pre_dispatch=pre_dispatch,
                         error_score=error_score, return_train_score=return_train_score)

        self.algorithm = algorithm
        self.cv = cv
        self.max_n_gen = max_n_gen
        self.max_stagnating_gen = max_stagnating_gen
        self.param_grid = param_grid
        self.population_size = population_size
        self.random_state = random_state
        self.runs = runs
        self.verbose = verbose
        self.optimization_logs_ = None
        self.cv_orig = None
        self.n_splits = None

    def __print_run_search_log(self):
        candidates = self.__param_grid.get_number_of_candidates()

        if self.cv is not None:
            print(f'Fitting {self.cv} folds for some of the {candidates} candidates, '
                  f'which might total in {candidates * self.n_splits} fits')
        else:
            print(f'Fitting at most {candidates} candidates')
